In [ ]:
import holodeck as holo
from holodeck import single_sources, utils
import holodeck.detstats as ds
from holodeck.constants import YR, MSOL, NWTG, SPLC

import numpy as np
import matplotlib.pyplot as plt
import kalepy as kale

In [ ]:
mtot=(1.0e6*MSOL/1.988409870698051, 1.0e8*MSOL/1.988409870698051, 3)
mrat=(1e-1, 1.0, 2)
redz=(1e-3, 1.0, 4)
sam = holo.sam.Semi_Analytic_Model(mtot=mtot, mrat=mrat, 
        redz=redz)
print('edges:', sam.edges)

In [ ]:
dur = 5.0*YR/3.1557600
cad = .5*YR/3.1557600
fo_gw = utils.nyquist_freqs(dur,cad)
fo_gw_edges = utils.nyquist_freqs_edges(dur,cad)
fo_orb_edges = fo_gw_edges / 2.0
fo_orb = fo_gw / 2.0

# dynamic_binary_number
# gets differential number of binaries per bin-vol per log freq interval
hard=holo.hardening.Hard_GW
edges, dnum = sam.dynamic_binary_number(hard=hard, fobs_orb=fo_orb)
edges[-1] = fo_orb_edges

# integrate (multiply by bin volume) within each bin
number = utils._integrate_grid_differential_number(edges, dnum, freq=False)
number = number * np.diff(np.log(fo_gw_edges))

# Dimensionless Amplitude of the signal

In [ ]:
print('G = %.2e cm^3 g^-1 s^-2' % NWTG)
tot_mass = kale.utils.midpoints(np.geomspace(mtot[0], mtot[1], mtot[2]))
mass_rat = kale.utils.midpoints(np.geomspace(mrat[0], mrat[1], mrat[2]))
chirp_mass = utils.chirp_mass_mtmr(tot_mass[:,np.newaxis], mass_rat[np.newaxis,:])
print('M_c:', chirp_mass.shape, chirp_mass, 'g')
redshift = kale.utils.midpoints(np.geomspace(redz[0], redz[1], redz[2]))
print('z:', redshift.shape, redshift)
comdist = holo.cosmo.comoving_distance(redshift).cgs.value
print('d_c:', comdist.shape, comdist, 'cm')

$$ A = 2 \frac{G^{5/3}\mathcal{M}^{5/3}[\pi f_{o,gw}[1+z]]^{2/3}}{c^4 r} \quad\quad (5)$$ 
in Rosado 2015

$\mathcal{M}$ is the proper chirp mass and $r$ is the comoving distance

In [ ]:
def amp_from_pars(mc, rz, fo_gw):
    """
    Parameters
    ----------
    mc : (M,Q)
        Proper chirp mass (rest frame).
    rz : (Z,)
        Redshift.
    fo_gw : (Z,F)
        Observed gw frequency.
    """
    mc = mc[:,:,np.newaxis,np.newaxis]
    rz = rz[np.newaxis,np.newaxis,:,np.newaxis]
    fo_gw = fo_gw[np.newaxis,np.newaxis,np.newaxis,:]
    dc = holo.cosmo.comoving_distance(rz).cgs.value
    numer = NWTG**(5/3) * mc**(5/3) * (np.pi * fo_gw*(1+rz))**(2/3)
    denom = SPLC**4 * dc
    amp = 2*numer/denom
    return amp

## Check units
* [G] = cm^3 g^-1 s^-2
* [M] = g
* [f] = s^-1
* [c] = cm s^-1
* [r] = cm

[G]^(5/3) [M] ^(5/3) [f]^(2/3) [c]^(-4) [r]^(-1) 

= (cm^3 g^-1 s^-2)^(5/3) (g)^(5/3) (s^-1)^(2/3) (cm s^-1)^(-4) (cm)^(-1) 

= cm^5 g^(-5/3) s^(-10/3) g^(5/3) s^(-2/3) cm^(-4) s^(4) cm^(-1)

= cm^5 cm^(-4) cm^(-1) g^(-5/3) g^(5/3) s^(-10/3)  s^(-2/3) s^(4) 

= cm^0 g^0 s^0 = dimensionless (check!)

In [ ]:
par_amp = amp_from_pars(chirp_mass, redshift, fo_gw)
print('A', par_amp.shape)
print('A[0,0,0]:', par_amp[0,0,0])

In [ ]:
hc2 = holo.gravwaves.char_strain_sq_from_bin_edges(edges)
print(hc2.shape)
hc = np.sqrt(hc2)
print(hc.shape)

In [ ]:
df = np.diff(fo_orb_edges)
print(df.shape)

hs = utils.char_strain_to_strain_amp(hc, fo_orb[np.newaxis,np.newaxis,:], 
                                     df[np.newaxis,np.newaxis,:]) # shape M,Z,F


In [ ]:
print(hs.shape, par_amp.shape)
print(hs[0,0,0])
print(par_amp[0,0,0])

In [ ]:
def hs_from_pars(mc, rz, fo_gw):
    """
    Parameters
    ----------
    mc : (M,Q)
        Proper chirp mass (rest frame).
    rz : (Z,)
        Redshift.
    fo_gw : (Z,F)
        Observed gw frequency.
    """
    mc = mc[:,:,np.newaxis,np.newaxis]
    rz = rz[np.newaxis,np.newaxis,:,np.newaxis]
    fo_gw = fo_gw[np.newaxis,np.newaxis,np.newaxis,:]
    fr_gw = utils.frst_from_fobs(fo_gw, rz) # rest frame gw frequency
    dc = holo.cosmo.comoving_distance(rz).cgs.value
    numer = 8*NWTG**(5/3)*mc**(5/3) * (np.pi*fr_gw)**(2/3)
    denom = np.sqrt(10) * SPLC**4 * dc
    return numer/denom

In [ ]:
par_hs = hs_from_pars(chirp_mass, redshift, fo_gw)
print(par_hs.shape)
print(par_hs[0,0,0], hs[0,0,0])

$h_s = A * \mathrm{factor}$

In [ ]:
plt.plot(np.arange(par_hs.size), (par_hs / par_amp).flatten())
print(4/np.sqrt(10))
factor = par_hs/par_amp
print(factor)

In [ ]:
plt.plot(np.arange(par_hs.size), ((10**(1/2)/4) * par_hs / par_amp).flatten(),
         label = r'$\frac{10^{1/2}}{4} \frac{h_s}{A}$')
plt.legend(fontsize=18)


In [ ]:
fr_orb = utils.frst_from_fobs(fo_orb[np.newaxis,:], 
                              redshift[:,np.newaxis]) # (Z,F)

hs_utils = utils.gw_strain_source(chirp_mass[:,np.newaxis,np.newaxis],
                                  comdist[np.newaxis,:,np.newaxis], 
                                  fr_orb[np.newaxis,:,:])
print(hs_utils.shape)

In [ ]:
print(par_hs.shape)
plt.plot(np.arange(par_hs.size), hs_utils.flatten(), alpha=0.5,
         label='hs from utils.gw_strain_source')
plt.plot(np.arange(par_hs.size), par_hs.flatten(), alpha=0.5,
         label='hs from pars')
plt.plot(np.arange(par_hs.size), hs.flatten(), alpha=0.5,
         label='hs from hc')
plt.legend()
plt.ylabel('hs')
plt.xlabel('ii')

In [ ]:
print(par_hs.shape)
plt.plot(fo_orb, hs_utils[0,0,0,:], alpha=0.5,
         label='hs from utils.gw_strain_source')
plt.plot(fo_orb, par_hs[0,0,0,:], alpha=0.5,
         label='hs from pars', linestyle='--')
plt.plot(fo_orb, hs[0,0,0,:], alpha=0.5,
         label='hs from hc')

dfo_orb = np.diff(fo_orb_edges)
plt.plot(fo_orb, hc[0,0,0,:]*np.sqrt(dfo_orb/fo_orb), alpha=0.5,
         label='hc*$\sqrt{df/f}$', linestyle='--')

plt.plot(fo_orb, par_amp[0,0,0,:] * 4/np.sqrt(10), alpha=0.5, linestyle=':',
         color='k', label = r'$\frac{4}{\sqrt{10}} A$')
plt.legend(fontsize=12)
plt.xlabel('fobs_orb')
plt.ylabel('hs[M=0, Z=0, :]')

# Everything works out!!

So in utils.char_strain_to_strain_amp we do $h_s = h_c * \sqrt{\Delta f_{obs,orb} / f_{obs,orb}}$ (which ends up not actually being needed)

In detstats._amplitude we do $A = \frac{10^{1/2}}{4} h_s = \frac{10^{1/2}}{4} h_c * \sqrt{\Delta f_{obs} / f_{obs}} $ where it doesnt matter if orbital or gw frequencies are used, as long as they are the same for $f_{obs}$ and $\Delta f_{obs}$